# Pregunta 1

* **a) (20 pts.)** En un curso de desarrollo de software se arman equipos de *k* alumnos que trabajan para un cliente. Cada grupo de alumnos es supervisado periódicamente por un ayudante y el profesor del curso. Cada alumno tiene una probabilidad de encontrarse enojado y una probabilidad de renunciar. Además, un alumno puede programar un cierto número de líneas de código por cada tarea que se le encomienda, que depende del lenguaje:

	* **Python:** Random entre 1 y 600 líneas
	* **JavaScript:** Random entre 1 y 400 líneas
	* **C:** Random entre 1 y 200 líneas

  Dentro del equipo hay un alumno especial que nunca renuncia, el jefe de grupo. Tanto el profesor como el ayudante pueden realizar comentarios. Cada uno de ellos tiene una probabilidad de decir un comentario positivo (y en caso contrario, un comentario negativo). Cada ayudante tiene una probabilidad de solucionar un conflicto del grupo que supervisa. Mientras tanto, el cliente tiene una probabilidad de máximo un 10% de hacer *boicot* al proyecto, ya sea porque el resultado no le gustó, o porque es tan bueno que no lo puede entender. Modele esta situación con OOP implementando las clases necesarias en Python.



* **b) (10 pts.)** **En el mismo contexto anterior**, cada semana hay una reunión del equipo junto al profesor y el ayudante. En esta oportunidad, el ayudante y el profesor emiten un comentario, que puede ser contradictorio (uno positivo y el otro negativo). Durante el desarrollo de la tarea, hay un conflicto si es que dos o más de los integrantes involucrados están enojados con probabilidad *p*. El ayudante puede solucionar con probabilidad *q* el conflicto del equipo por reunión. Existe una reunión mensual del equipo junto al cliente, donde el cliente podría *boicotear* el trabajo realizado.

  Cada cierto tiempo aleatorio (entre 0 y 7 días), parte del equipo se junta para realizar una tarea. Al inicio de ésta, se elige un lenguaje al azar entre `Python`, `JavaScript` o `C` para trabajar. Las líneas de código escritas corresponden a la suma de lo que cada integrante puede escribir en ese lenguaje. Además, es posible que uno o más integrantes decidan renunciar.

  Usted debe generar 20 simulaciones en paralelo con threads, para 3 meses y un grupo de 9 personas. Ocupe las clases implementadas en la parte a). Para cada simulación, debe extraer el número de veces que el ayudante se contradijo con el profesor, el número de tareas realizadas por el equipo, el número de líneas de código escritas en total, la cantidad de conflictos que ocurrieron, la cantidad de conflictos solucionados, el número de integrantes que renunció y el número de veces que el cliente intentó *boicotear* el proyecto. **HINT:** `rand() < p` retorna `True` con probabilidad *p*

In [1]:
from abc import ABCMeta
from random import choice, randint, random, shuffle
from threading import currentThread, Thread

## Traduciendo el enunciado al modelo

Del enunciado podemos obtener directamente muchas clases que tendremos que crear:

* Curso
* Equipo
* Alumno
* Implementador
* Ayudante
* Profesor
* Jefe de grupo
* Cliente

Hay algunas clases que deben ser abstractas porque solo instanciaremos clases que hereden de estas: Alumno y Docente. La clase que controlará la simulación será Curso y la clase que representará cada thread en la simulación es Equipo.

### Alumno

Tendremos una clase Alumno (abstracta), Jefe (porque es un alumno especial) e Implementador (ya que se debe diferenciar a un Alumno del Jefe)

In [2]:
class Alumno(metaclass=ABCMeta):
    
    @classmethod
    def factory(cls):
        return cls({'python': randint(1, 600), 'javascript': randint(1, 400), 'c': randint(1, 200)},
                   random(),
                   random())
    
    def __init__(self, habilidad_lenguaje, prob_enojo, prob_renuncia):
        self.habilidad_lenguaje = habilidad_lenguaje
        self.prob_enojo = prob_enojo
        self.prob_renuncia = prob_renuncia
        
    def programar_en(self, lenguaje):
        return self.habilidad_lenguaje[lenguaje]
    
    @property
    def enojado(self):
        return random() < self.prob_enojo
    
    @property
    def renuncia(self):
        return random() < self.prob_renuncia
    
class Jefe(Alumno):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    @property
    def renuncia(self):
        return False
    
class Implementador(Alumno):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

### Cliente 

Sabemos que el cliente puede boicotear nuestro proyecto

In [3]:
class Cliente:
    
    @classmethod
    def factory(cls):
        return cls(random())
    
    def __init__(self, prob_boicot):
        self.prob_boicot = prob_boicot
        
    @property
    def boicotea(self):
        return random() < self.prob_boicot

### Docentes

Profesor y Ayudante tienen comportamientos parecidos, ambos emiten comentarios. Pero está mal modelado el que Ayudante herede de Profesor. ¿Por qué? Porque un Ayudante no es un Profesor. Si Profesor tuviese el poder de sacar a un alumno del curso, ¿el ayudante también debería poder hacer esto? No, porque un ayudante no es un profesor.

In [4]:
class Docente(metaclass=ABCMeta):
    
    def __init__(self, prob_comentario_positivo):
        self.prob_comentario_positivo = prob_comentario_positivo
        
    @property
    def comenta_positivamente(self):
        return random() < self.prob_comentario_positivo
    
class Profesor(Docente):
    
    @classmethod
    def factory(cls):
        return cls(random())
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
class Ayudante(Docente):
    
    @classmethod
    def factory(cls):
        return cls(random(), random())
    
    def __init__(self, prob_comentario_positivo, prob_solucionar_conflicto):
        self.prob_comentario_positivo = prob_comentario_positivo
        self.prob_solucionar_conflicto = prob_solucionar_conflicto
        
    @property
    def soluciona_conflicto(self):
        return random() < self.prob_solucionar_conflicto

### Grupo

Aquí va la simulación, ya que tenemos que simular el comportamiento de cada grupo, por lo que cada grupo es un thread

In [5]:
class Grupo(Thread):
    
    def __init__(self, tamano=9, profesor=None, limite=7*4*3):
        super().__init__()
        # Instancias       
        self.ayudante = Ayudante.factory()
        self.cliente = Cliente.factory()
        self.implementadores = [Implementador.factory()
                               for _ in range(tamano - 1)]
        self.jefe = Jefe.factory()
        self.profesor = profesor or Profesor.factory()
        # Tiempos
        self.tiempo_actual = 0
        self.tiempo_limite = limite        
        self.tiempo_proxima_reunion_cliente = 0
        self.tiempo_proxima_reunion_docentes = 0
        self.tiempo_proxima_reunion_tarea = 0
        # Variables
        self.hay_conflicto = False
        # Indicadores
        self.contradicciones = 0
        self.tareas_realizadas = 0
        self.lineas_escritas = 0
        self.conflictos_ocurridos = 0
        self.conflictos_solucionados = 0
        self.integrantes_que_renunciaron = 0
        self.intentos_de_boicoteo = 0
        
    def run(self):
        while self.tiempo_actual < self.tiempo_limite:
            if self.tiempo_actual == self.tiempo_proxima_reunion_cliente:
                self.reunion_cliente()
            if self.tiempo_actual == self.tiempo_proxima_reunion_docentes:
                self.reunion_docentes()
            if self.tiempo_actual == self.tiempo_proxima_reunion_tarea:
                self.reunion_tarea()
            self.tiempo_actual += 1
        self.__imprimir_indicadores()
        
    def reunion_cliente(self):
        
        # Boicot de cliente
        if self.cliente.boicotea:
            self.intentos_de_boicoteo += 1
            
        self.__proxima_reunion_cliente()
        
    def reunion_docentes(self):
        
        # Comentarios de profesor y ayudante
        comentario_profesor = self.profesor.comenta_positivamente
        comentario_ayudante = self.ayudante.comenta_positivamente
        if comentario_profesor != comentario_ayudante:
            self.contradicciones += 1
            
        # Solucion de conflictos
        if self.hay_conflicto and self.ayudante.soluciona_conflicto:
            self.conflictos_solucionados += 1
            self.hay_conflicto = False
            
        self.__proxima_reunion_docentes()
        
    def reunion_tarea(self):
        self.tareas_realizadas += 1
        
        # Conflictos
        if self.__revisar_conflictos():
            self.hay_conflicto = True
            self.conflictos_ocurridos += 1
            
        # Programar
        lenguaje = choice(["python", "javascript", "c"])
        self.lineas_escritas += sum([alumno.programar_en(lenguaje) for alumno in self.todos])

        # Renuncias
        renuncias = [alumno for alumno in self.implementadores if alumno.renuncia]
        for alumno in renuncias:
            self.integrantes_que_renunciaron += 1
            self.implementadores.remove(alumno)
            
        self.__proxima_reunion_tarea()
        
    @property
    def todos(self):
        return [self.jefe] + self.implementadores
            
    def __revisar_conflictos(self):
            return len(
                    list(
                        filter(
                            lambda alumno: alumno.enojado, 
                            self.todos
                        )
                    )
                ) >= 2           
        
    def __proxima_reunion_cliente(self):
        self.tiempo_proxima_reunion_cliente += 7 * 4
    
    def __proxima_reunion_docentes(self):
        self.tiempo_proxima_reunion_docentes += 7
    
    def __proxima_reunion_tarea(self):
        self.tiempo_proxima_reunion_tarea += randint(1, 7)
        
    def __imprimir_indicadores(self):
        # Thread actual
        print(currentThread().getName())
        # Indicadores y valores
        print("Contradicciones entre el Ayudante y el profesor: {}".format(
            self.contradicciones))
        print("Tareas realizadas: {}".format(
                self.tareas_realizadas))
        print("Líneas escritas en total: {}".format(
                self.lineas_escritas))
        print("Conflictos solucionados / total: {} / {}".format(
            self.conflictos_solucionados, self.conflictos_ocurridos))
        print("Burn outs del equipo: {}".format(
                self.integrantes_que_renunciaron))
        print("Boicots del cliente: {}".format(
                self.intentos_de_boicoteo))
        print()

### Curso

Esta clase controla la simulación y contiene a todos los grupos

In [6]:
class Curso:
    
    def __init__(self, tamano_equipo=9, numero_equipos=20):
        self.profesor = Profesor.factory()
        self.equipos = [Grupo(tamano=tamano_equipo, profesor=self.profesor)
                       for _ in range(numero_equipos)]
        
    def simular(self):
        for equipo in self.equipos:
            equipo.start()

## Output

In [7]:
curso = Curso()
curso.simular()

Thread-6
Contradicciones entre el Ayudante y el profesor: 3
Tareas realizadas: 21
Líneas escritas en total: 16996
Conflictos solucionados / total: 2 / 3
Burn outs del equipo: 7
Boicots del cliente: 0

Thread-7
Contradicciones entre el Ayudante y el profesor: 7
Tareas realizadas: 19
Líneas escritas en total: 7640
Conflictos solucionados / total: 0 / 3
Burn outs del equipo: 8
Boicots del cliente: 0

Thread-8
Contradicciones entre el Ayudante y el profesor: 7
Tareas realizadas: 21
Líneas escritas en total: 4005
Conflictos solucionados / total: 1 / 3
Burn outs del equipo: 8
Boicots del cliente: 1

Thread-9
Contradicciones entre el Ayudante y el profesor: 4
Tareas realizadas: 23
Líneas escritas en total: 6719
Conflictos solucionados / total: 2 / 6
Burn outs del equipo: 8
Boicots del cliente: 2

Thread-10
Contradicciones entre el Ayudante y el profesor: 1
Tareas realizadas: 19
Líneas escritas en total: 9956
Conflictos solucionados / total: 1 / 1
Burn outs del equipo: 8
Boicots del cliente: 2

## Distribucion de puntaje

### Parte A (20 puntos)

| Puntaje | Explicación |
| :-: | :-- |
| 2.0 | Por cada clase implementada correctamente (son 8 clases)|
| 1.0 | Por cada herencia correcta en el modelamiento (son 4 herencias) |

### Parte B (10 puntos)

| Puntaje | Explicación |
| :-: | :-- |
| 2.0 | Correcto uso de Threads |
| 1.0 | Output de la simulación |
| 1.0 | Por cada indicador calculado correctamente (son 7 indicadores) |

### Notas

* Los métodos de clase `factory()` fueron agregados solamente para facilitar la escritura del programa, no se esperaba que los escribieran ustedes en sus respuestas.
* Se descontaron puntos en los casos en que el modelamiento no apuntaba a lo esperado
* No se asignó puntaje a crear clases que no eran necesarias, por ejemplo, crear una clase Tarea